In [18]:
import psycopg2
import pandas as pd
import csv
import time
import os
import openai
import numpy as np

# filepath ='/home/azureuser/cloudfiles/code/Users/supakit/SimilaritySearch/Mon/'

In [19]:
openai.api_type = "azure"
openai.api_base = "https://fusion-chatbot-demo.openai.azure.com/"
openai.api_version = "2023-03-15-preview"
openai.api_key = "f07d23e10600494f8235496bbf5054b9"

In [20]:
#production
host = "fusion-postgresql-prd.postgres.database.azure.com"
dbname = "postgres"
user = "pgadmin"
password = "Fusi@nS@luti@nVV9"
sslmode = "require"
# Construct connection string

In [21]:
conn_string = "host={0} user={1} dbname={2} password={3} sslmode={4}".format(host, user, dbname, password, sslmode)
conn = psycopg2.connect(conn_string) 
print("Connection established")
cursor = conn.cursor()
# Fetch all rows from table

Connection established


In [22]:
import requests, uuid, json

# Add your key and endpoint
key = "041e6b79b96146eabafaf524d78911be"
endpoint = "https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = "southeastasia"
path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'th',
    'to': ['en']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
def translator(input):
    # translated = []
    # for string in input:
    body = [{
        'text': input
    }]
    request = requests.post(constructed_url, params=params, headers=headers, json=body)
    response = request.json()
    # translated.append(response[0]["translations"][0]['text'])
    # print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))
    # print(response[0]["translations"][0]['text'])
    return response[0]["translations"][0]['text']


In [23]:
def text2vec(input):
    input = translator(input)
    embedding = openai.Embedding.create(input = input, engine = "text-embedding-ada-002")
    return embedding['data'][0]['embedding']

def search_vectordb(question):
    search_text = []
    try:
        cursor.execute(f"""SELECT text_org,text_en , cosine_distance('{text2vec(question)}', embedding) as cosign
        FROM tblvector
        WHERE tblvector.vectordbname = 'ThaiLife'
        ORDER BY cosine_distance('{text2vec(question)}', embedding) ASC LIMIT 3
        """)
        query = cursor.fetchall()
        return query
    except Exception as e:  
        print(f"Error: {e}")
        conn.rollback()
        return "null"

#WHERE cosine_distance('{text2vec(question)}', embedding) < {start}

In [24]:
prompt='''
You are an Insurance Consultant bot of Thai Life Insurance, an insurance company in Thailand. Your main responsibility is to address the countless insurance plan questions you receive from customers on a daily basis. Your objective is to provide answers for customers based on the provided insurance plan. The question that you have received may be related to 1 or multiple plans.  Your response will be strictly under these rules:
1. You will only recommend the insurance plan of Thai Life Insurance and you will not recommend customer another Insurance Company's Plan that is not Thai Life.
2. Your response will be as short, clear, and concise as possible you can do.
3. Avoid repeating the question and focus solely on providing a plan and answering the question related to the provided insurance plan only.
4. Please make sure that when you received a question in Thai, your response will also be in Thai.
5. If you don’t know the answer or feel that the answer might lead to the problem, please do not try to make it up and give Customer Services contact of Thai Life only. 
6. If a customer ask you to recommend an insurance plan, please respond in this format

	แบบประกัน: 
	จุดเด่นของประกัน:
7. If customers ask you about individual plans you will respond short and concisely as much as possible.		
8. To clarify, if you encounter any question that is not about the insurance plan, you will only response (" ฉันไม่มีข้Fอมูลเกี่ยวกับคำถามนี้") and give them Thai Life Customer Service contact.

Thai Life Customer Services contact;
- Tel.1124 ( Services are provided every day from 8:00 - 21:00)
- Email: carecenter@thailife.com
- Automated phone response system is available 24 hours a day
	Press 1 for inquiries regarding insurance policies.
	Press 2 for inquiries regarding Thai Life Insurance services.
	Press 3 for inquiries regarding policies for กบข. members.
	Press 4 for inquiries regarding activities.


#These are the provided insurance plan
'''

In [8]:
#Generate Response with searched_vector in csv
import os
import csv
import time

def generate_response(input_file, prompt, OpenAI_engine, name_after, token, times):
    input_filename, input_extension = os.path.splitext(input_file)
    output_file = input_filename + str(OpenAI_engine) + str(name)_after + "_out.csv"

    questions = []
    with open(input_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            questions.append(row[0])  # Assuming the questions are in the first column

    # Generate responses for each question and write to CSV
    output_rows = []
    for i in range(len(questions)):
        question = questions[i]
        search_fact = str(search_vectordb(question))
        
        try:
            response = openai.ChatCompletion.create(
                engine=OpenAI_engine,
                messages=[
                    {"role": "system", "content": prompt + search_fact},
                    {"role": "user", "content": question}
                ],
                temperature=0,
                max_tokens=token,
                top_p=0,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None
            )

            if 'choices' in response:
                # Extract the answer from the response
                answer = response['choices'][0]['message']['content']

                # Append question and answer to output rows
                output_rows.append([question, answer, search_fact])

                print(answer)

                # Pause for 5 seconds before making the next API request
                time.sleep(times)

            elif 'error' in response and response['error']['code'] == 'max_tokens':
                # Handle token error by outputting "Token Error" for the question
                print("Token Error")
                output_rows.append([question, "Token Error", search_fact])
                
            else:
                # Handle other errors by outputting "Error" for the question
                print("Error occurred for question:", question)
                output_rows.append([question, "Error", search_fact])

        except Exception as e:
            # Handle any other exceptions by outputting "Error" for the question
            print("Exception occurred for question:", question)
            print(str(e))
            output_rows.append([question, "Error", search_fact])

    # Write output rows to the generated output file
    with open(output_file, 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(output_rows)

In [9]:
##Generate response without searched vector in csv
import os
import csv
import time

def generate_response_nosearch(input_file, prompt, OpenAI_engine, version, token, times):
    input_filename, input_extension = os.path.splitext(input_file)
    output_file = input_filename + str(OpenAI_engine) + str(version) + "_out.csv"

    questions = []
    with open(input_file, 'r', encoding='utf-8') as f:
        reader = csv.reader(f)
        for row in reader:
            questions.append(row[1])  # Assuming the questions are in the first column

    # Generate responses for each question and write to CSV
    output_rows = []
    for i in range(len(questions)):
        question = questions[i]
        search_fact = str(search_vectordb(question))
        
        try:
            response = openai.ChatCompletion.create(
                engine=OpenAI_engine,
                messages=[
                    {"role": "system", "content": prompt + search_fact},
                    {"role": "user", "content": question}
                ],
                temperature=0,
                max_tokens=token,
                top_p=0,
                frequency_penalty=0,
                presence_penalty=0,
                stop=None
            )

            if 'choices' in response:
                # Extract the answer from the response
                answer = response['choices'][0]['message']['content']

                # Append question and answer to output rows
                output_rows.append([question, answer])

                print(answer)

                # Pause for 5 seconds before making the next API request
                time.sleep(times)

            elif 'error' in response and response['error']['code'] == 'max_tokens':
                # Handle token error by outputting "Token Error" for the question
                print("Token Error")
                output_rows.append([question, "Token Error"])
                
            else:
                # Handle other errors by outputting "Error" for the question
                print("Error occurred for question:", question)
                output_rows.append([question, "Error"])

        except Exception as e:
            # Handle any other exceptions by outputting "Error" for the question
            print("Exception occurred for question:", question)
            print(str(e))
            output_rows.append([question, "Error"])

    # Write output rows to the generated output file
    with open(output_file, 'w', encoding='utf-8-sig', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(output_rows)

In [10]:
#Generate only 1 response
def generate_one_response(test_question,prompt,OpenAI_engine,token):
    search_fact = str(search_vectordb(test_question))
    response = openai.ChatCompletion.create(
        engine=OpenAI_engine,
        messages=[
            {"role": "system", "content": prompt+search_fact},
            {"role": "user", "content": test_question}
        ],
        temperature=0,
        max_tokens=token,
        top_p=0,
        frequency_penalty=0,
        presence_penalty=0,
        stop=None
    )
    # Extract the answer from the response
    answer = response['choices'][0]['message']['content']
    print(answer)
    #print(search_fact)

In [11]:
test_question = "ในแผนปรกันเติมเงินยามป่วย ผู้เอาประกันอายุเท่าใดสามารถเอาประกันได้"
prompt = prompt
OpenAI_engine = 'gpt-4'
token = 2000
generate_one_response(test_question,prompt,OpenAI_engine,token)

แผนประกัน "เติมเงินยามป่วย" สามารถสมัครได้สำหรับผู้ที่อายุระหว่าง 26-50 ปีค่ะ.


In [21]:
input_file ='Pos_Gen.csv'
OpenAI_engine ='gpt-35-turbo'
name_after  ='result_35_Turbo'
token = 700
times = 10
generate_response(input_file, prompt,OpenAI_engine,Name_After,token,times)

ค่าชดเชยรายวันสำหรับการรักษาในโรงพยาบาลสูงสุดของแผนประกัน Smile Kids คือ 3000 บาทต่อวันค่ะ
แผนประกัน Smile Kids มีการชดเชยค่าใช้จ่ายสำหรับโรคที่พบบ่อยในเด็ก 5 โรค ได้แก่ โรคปอดบวม หรือ ปอดอักเสบ, โรคมือเท้าปาก, โรคอาหารเป็นพิษ, โรคบาดทะยัก และโรคไข้สมองอักเสบจากเชื้อไวรัส
ผู้เอาประกันภัยจะได้รับเงินคืนสูงสุดถึง 40,000 บาทในวันครบรอบ 8 ปีของกรมธรรม์ Smile Kids
ผู้เอาประกันภัยจะได้รับเงินคืนสูงสุดถึง 40000 บาท ในสิ้นสุดของสัญญาแผนประกัน Smile Kids
Exception occurred for question:  แผนประกัน Smile Kids นี้เหมาะสำหรับผู้ที่ต้องการประกันสุขภาพให้กับลูกหลานของพวกเขาหรือไม่?
This model's maximum context length is 4096 tokens. However, you requested 4195 tokens (3495 in the messages, 700 in the completion). Please reduce the length of the messages or completion.
แผนประกัน Smile Kids มีการคุ้มครองค่าใช้จ่ายทางการแพทย์ในกรณีเกิดอุบัติเหตุครับ
แผนประกัน Smile Kids จะมอบความคุ้มครองให้กับผู้ที่มีความกังวลเกี่ยวกับการเจ็บป่วยของลูก ๆ ดังนี้:
1. ค่าชดเชยรายวัน กรณีเข้าพักรักษาตัวในโรงพยาบาลสูงสุดวัน

คุณจะได้รับเงินคืนทุกปี 1% ของจำนวนเงินเอาประกันภัยในแผนประกันภัย ออมยาวคุ้มทวีคูณ ของคุณครับ
ในสิ้นสุดของสัญญาแผนประกันภัย ออมยาวคุ้มทวีคูณ คุณจะได้รับเงินคืนเท่ากับ 110% ของเบี้ยประกันภัยที่คุณชำระให้แก่บริษัทฯ ทั้งนี้ ไม่รวมเบี้ยประกันภัยสัญญา (ถ้ามี) หักด้วยผลประโยชน์เงินคืนระหว่างสัญญา และไม่รวมเงินคืนที่ได้รับในทุกปี ซึ่งมีอัตรา 1% ของเบี้ยประกันภัย (ตั้งแต่สิ้นปีกรมธรรม์ที่ 1 จนถึงปีก่อนครบกำหนดสัญญา)
คุณสามารถหักภาษีได้สูงสุด 100,000 บาท จากแผนประกันภัย ออมยาวคุ้มทวีคูณ ของเราค่ะ
แผนประกันภัยออมยาวคุ้มทวีคูณเหมาะสำหรับผู้ที่ต้องการประกันชีวิตที่มั่นคง โดยท่านจะได้รับการคุ้มครองชีวิตจนถึงอายุ 85 ปี และสามารถลดหย่อนภาษีได้สูงสุด 100,000 บาทต่อปี นอกจากนี้ยังมีการรับเงินคืนทุกปี ปีละ 1% ของจำนวนเงินเอาประกันภัย และครบกำหนดสัญญารับ 110% ของจำนวนเงินเอาประกันภัย ดังนั้นแผนประกันภัยนี้จึงเหมาะสำหรับผู้ที่ต้องการมีหลักประกันที่มั่นคงในชีวิตและใช้สิทธิลดหย่อนภาษีได้ด้วยค่ะ
แน่นอนครับ แผนประกันภัยออมยาวคุ้มทวีคูณของเราเหมาะสำหรับผู้ที่ต้องการใช้สิทธิลดหย่อนภาษี โดยสามารถลดหย่อนภาษีได้สู

แผนประกันภัยเติมเงินยามป่วยของเราจะชดเชยรายได้เมื่อผู้เอาประกันภัยต้องรับการรักษาในโรงพยาบาลเนื่องจากโรคติดเชื้อ COVID-19 ได้สูงสุดวันละ 1,500 บาท ตลอด 365 วัน
แบบประกันภัยเติมเงินยามป่วยเหมาะสำหรับกลุ่มคนที่ต้องการความคุ้มครองเมื่อเจ็บป่วย พนักงานบริษัท และผู้ประกอบอาชีพอิสระ
แผนประกันภัย เติมเงินยามป่วย จะชดเชยค่ารักษาพยาบาลเท่ากับ 1500 บาทต่อวัน ในฐานะผู้ป่วยใน ไม่ว่าจะป่วยจากการติดเชื้อ COVID-19 ป่วยไข้หรืออุบัติเหตุ สูงสุด 365 วัน
แผนประกันภัย เติมเงินยามป่วย จะได้รับเงินสดคืน 1500 บาท หลังจากที่ออกจากโรงพยาบาลครั้งละครบกำหนดสัญญา
แผนประกันภัยเติมเงินยามป่วยของเรามีการคุ้มครองสูงสุดวันละ 1,500 บาท ในกรณีที่ผู้เอาประกันภัยต้องรับการรักษาในโรงพยาบาล โดยไม่ว่าจะเป็นการป่วยจากการติดเชื้อ COVID-19 หรือป่วยไข้หรืออุบัติเหตุ สูงสุด 365 วัน
แผนประกันภัยเติมเงินยามป่วยเหมาะสำหรับช่วงอายุ 26-50 ปีค่ะ
แผนประกันภัยเติมเงินยามป่วยของเราคุ้มครองสูงสุดครั้งละ 30,000 บาทในกรณีที่ผู้เอาประกันภัยต้องรับการผ่าตัดใหญ่ครั้งหนึ่งครั้ง
แผนประกันภัยเติมเงินยามป่วยของเราจะชดเชยรายได้เมื่อผู้เอาประกันภัยต้

ค่าเบี้ยประกันของแผนประกันมรดกเพิ่มพูนถูกคำนวณจากเงื่อนไขต่างๆ เช่น อายุของผู้เอาประกันภัย จำนวนเงินเอาประกันภัย และระยะเวลาของสัญญา โดยค่าเบี้ยประกันจะเริ่มต้นที่ 8 บาทต่อวัน โดยคำนวณจากเบี้ยประกันรายปี ของเพศหญิง อายุ 50 ปี และจำนวนเงินเอาประกันภัย 50000 บาท
แบบประกันทรัพย์มิ่งขวัญ ผู้เอาประกันภัยจะได้รับเงินคืน 150% เมื่อครบวันครบรอบประกันภัยที่อายุ 75 ปี
แผนประกันสูงวัยมีทรัพย์มีการคุ้มครองจากโรคและอุบัติเหตุสูงสุดถึง 600,000 บาทค่ะ
ในแผนประกันสูงวัยมีทรัพย์ ผู้เอาประกันภัยจะได้รับเงินคืนเต็มจำนวนเมื่อครบวันครบรอบประกันภัย สูงสุดถึง 600,000 บาท
แบบประกันภัยสูงวัยมีทรัพย์ของเรามีค่าเบี้ยเริ่มต้นเพียงวันละ 9 บาทครับ/ค่ะ
แผนประกัน "สูงวัยมีทรัพย์" สามารถสมัครได้ตั้งแต่อายุ 51-60 ปี ค่ะ
ไม่ต้องการการตรวจสุขภาพ
แผนประกัน "สูงวัยมีทรัพย์" เหมาะสำหรับกลุ่มผู้สูงอายุ โดยเฉพาะ ซึ่งจะมีความคุ้มครองที่ครอบคลุมตลอดชีวิต ไม่ว่าจะเป็นการคุ้มครองจากการเจ็บป่วยหรือจากอุบัติเหตุ และยังมีการคืนเงินคืนสูงถึง 600,000 บาท ในกรณีที่ไม่มีการเคลื่อนไหวในกรมธรรม์ นอกจากนี้ยังมีเบี้ยเริ่มต้นที่เพียง 9 บาทต่

แบบประกันที่เหมาะสำหรับพ่อแม่วัย 79 คือ "ทรัพย์มิ่งขวัญ 90/15" คุ้มครองถึงอายุ 90 ปี ครบสัญญารับเงินคืน 150% ไม่ต้องตรวจสุขภาพ แต่ต้องแถลงสุขภาพ สามารถลดหย่อนภาษีได้สูงสุด 100,000 บาท และสร้างกองทุนมรดกสำหรับบุตรหลาน หากคุณต้องการข้อมูลเพิ่มเติมหรือการสมัคร สามารถติดต่อฝ่ายบริการลูกค้าของเราได้ที่เบอร์โทร 1124 หรืออีเมล carecenter@thailife.com ขอบคุณครับ
แบบประกันที่คุณควรซื้อคือแบบประกัน "ก้าวแรก" ของ Thai Life Insurance ครับ

จุดเด่นของแบบประกันนี้คือ:
- รับประกันบุตรตั้งแต่แรกเกิดถึง 90 วัน สำหรับคุณแม่ที่มีกรมธรรม์ก้าวแรก
- เตรียมความพร้อมด้านทุนการศึกษาและสวัสดิการด้านสุขภาพให้แก่ลูก
- ระหว่างสัญญารับเงินคืน 10% ทุกวันครบรอบ 3 ปีกรมธรรม์
- ครบกำหนดสัญญารับ 150%
- ชำระเบี้ยฯ 10 ปี คุ้มครอง 15 ปี พร้อมลดหย่อนภาษีได้สูงสุด 100,000 บาท
- รับประกันมารดาอายุ 20 - 45 ปี

แบบประกันนี้เหมาะสำหรับ:
- ผู้ที่ต้องการวางแผนทุนการศึกษาสำหรับบุตร
- ผู้ที่ต้องการความคุ้มครองในช่วงเริ่มต้นของชีวิตครอบครัว

หากคุณต้องการข้อมูลเพิ่มเติมหรือมีคำถามเกี่ยวกับแบบประกันนี้ คุณสามารถติดต่อศูนย์บริการลูกค้า

In [22]:
input_file ='Pos_Gen2.csv'
OpenAI_engine ='gpt-35-turbo'
name_after  ='result_35_Turbo'
token = 700
times = 10
generate_response(input_file, prompt,OpenAI_engine,name_after,token,times)

ค่าชดเชยรายวันสำหรับการรักษาในโรงพยาบาลสูงสุดของแผนประกัน Smile Kids คือ 3000 บาทต่อวันค่ะ
แผนประกัน Smile Kids จะชดเชยค่าใช้จ่ายในโรงพยาบาลประจำวันสูงสุดวันละ 3,000 บาทค่ะ
Exception occurred for question:  แผนประกัน Smile Kids ได้รับเงินคืนสูงสุดเท่าไหร่?
This model's maximum context length is 4096 tokens. However, you requested 4203 tokens (3503 in the messages, 700 in the completion). Please reduce the length of the messages or completion.
แผนประกัน Smile Kids ในสิ้นสุดของสัญญา ผู้เอาประกันภัยจะได้รับเงินคืนสูงสุด 174% ของจำนวนเงินเอาประกันภัย
Exception occurred for question:  แผนประกัน Smile Kids นี้เหมาะสำหรับผู้ที่ต้องการเตรียมเงินออมและรักษาสุขภาพของลูกหลานของพวกเขาหรือไม่?
This model's maximum context length is 4096 tokens. However, you requested 4207 tokens (3507 in the messages, 700 in the completion). Please reduce the length of the messages or completion.
Exception occurred for question: แผนประกัน Smile Kidsนี้เหมาะสำหรับผู้ใด
This model's maximum context length is 4096 tok

แผนประกันภัย P.A. Beyond Smart Protection สามารถนำมาหักภาษีได้ค่ะ สูงสุดถึง 100,000 บาทต่อปีค่ะ
แผนประกันภัย P.A. Beyond Smart Protection เหมาะสำหรับช่วงอายุ 0-25 ปีค่ะ
แผนประกันภัย P.A. Beyond Smart Protection จะคุ้มครองค่าใช้จ่ายในการรักษาทางการแพทย์จากอุบัติเหตุสูงสุด 50,000 บาทต่อครั้ง (1)
แผนประกันภัย P.A. Beyond Smart Protection มีการคุ้มครองอุบัติเหตุที่เกิดจากการขับขี่หรือโดยสารรถจักรยานยนต์สูงสุดถึง 250,000 บาทค่ะ
แผนประกัน P.A. Refund สามารถรับค่าชดเชยรายได้ระหว่างเข้ารักษาตัวในโรงพยาบาลวันละ 1,000 บาทได้สูงสุด
แผนประกัน P.A. Refund ของเราจะมีการคืนเงินให้ในปีสุดท้ายหากคุณไม่มีการเรียกร้องใด ๆ ในรอบ 3 ปี โดยจะคืนเงินให้เป็นจำนวนเงินที่คุณชำระเบี้ยประกันภัยในปีสุดท้ายของกรมธรรม์ โดยไม่รวมเบี้ยประกันภัยสัญญาอื่น ๆ ที่คุณอาจมีอยู่ หากมีเงินคืนจากการชำระเบี้ยประกันภัยในปีสุดท้ายที่มากกว่าเงินคืนจากการคืนเงินในรอบ 3 ปี บริษัทจะจ่ายเงินคืนให้เท่ากับจำนวนที่มากกว่านั้นให้แก่คุณ
แบบประกัน P.A. Refund ในกรณีที่คุณไม่สามารถทำงานและต้องพักผ่อนในโรงพยาบาล คุณจะได้รับค่าชดเชยสูงสุด 1,500 

แผนประกันภัยเฮลท์เหมา สบายใจ สบายตังค์ มีการคุ้มครองสูงสุดถึง 5 ล้านบาทต่อรอบปีของกรมธรรม์ค่ะ
แผนประกันภัย "เฮลท์เหมา สบายใจ สบายตังค์" ของเราจะคุ้มครองค่าใช้จ่ายทั้งหมดในกรณีที่เกิดอุบัติเหตุหรือป่วย ด้วยวงเงินค่ารักษาพยาบาลแบบเหมาจ่ายสูงสุดถึง 5 ล้านบาทต่อรอบปีกรมธรรม์ และไม่ต้องสำรองจ่ายสำหรับโรงพยาบาลตามคู่สัญญา ดังนั้นคุณสามารถมั่นใจได้ว่าค่าใช้จ่ายทั้งหมดจะได้รับความคุ้มครองจากแผนประกันภัยนี้ครอบคลุม
ไม่ต้องผ่านการตรวจสุขภาพค่ะ แค่ตอบคำถามสุขภาพ 2 ข้อเท่านั้น
Exception occurred for question:  ประกันภัย เฮลท์เหมา สบายใจ สบายตังค์ ในกรณีที่ต้องเข้ารับการรักษาในโรงพยาบาล ผู้เอาประกันภัยต้องจ่ายค่าใช้จ่ายล่วงหน้าหรือไม่?
This model's maximum context length is 4096 tokens. However, you requested 4140 tokens (3440 in the messages, 700 in the completion). Please reduce the length of the messages or completion.
แบบประกันภัยเฮลท์เหมา สบายใจ สบายตังค์ ของเราสามารถเลือกจำนวนเงินที่ต้องจ่ายเอง (deductible) ได้ค่ะ โดยสามารถเลือกความรับผิดส่วนแรก (deductible) ได้ตามต้องการของท่านเองค่ะ
แผนประก

แผนประกันโรคร้ายแรงรีฟันด์เหมาะสำหรับผู้ที่มีความกังวลเรื่องโรคร้ายแรงและต้องการใช้สิทธิลดหย่อนภาษี
แผนประกันโรคร้ายแรงรีฟันด์ของเราสามารถช่วยให้คุณได้รับประโยชน์ในการหักลดภาษีได้สูงสุดถึง 100,000 บาทต่อปี นอกจากนี้ยังมีจุดเด่นอื่น ๆ เช่น คุ้มครอง 10 โรคร้ายแรงในกลุ่มโรคมะเร็ง โรคหัวใจ และโรคหลอดเลือดในสมอง ตรวจพบโรคร้ายแรงรับเงินสูงสุด 1,000,000 บาท และนอนโรงพยาบาลรับเงินชดเชยวันละ 1,000 บาท สูงสุด 365 วัน หากคุณสนใจเพิ่มเติมหรือต้องการข้อมูลเพิ่มเติม กรุณาติดต่อฝ่ายบริการลูกค้าของเราที่เบอร์ 1124 ขอบคุณค่ะ
แผนประกันโรคร้ายแรงรีฟันด์เหมาะสำหรับช่วงอายุ 36-60 ปีค่ะ
แผนประกันโรคร้ายแรงรีฟันด์ของเราจะจ่ายค่าชดเชยวันละ 1,000 บาท เมื่อคุณเข้ารักษาในโรงพยาบาลในระยะเวลาเดียวกัน สูงสุด 365 วัน
แผนประกันโรคร้ายแรงรีฟันด์เหมาะสำหรับผู้ที่มีความกังวลเรื่องโรคร้ายแรงและต้องการความคุ้มครองในกรณีเกิดโรคมะเร็ง โรคหัวใจ หรือโรคหลอดเลือดในสมอง อายุระหว่าง 36-60 ปี
แผนประกันภัย Credit Care มีการคุ้มครองสูงสุดจากอุบัติเหตุถึง 1,000,000 บาทค่ะ
แผนประกันภัย Credit Care จะคุ้มครองการชดเชยสำหรับบัตรเครดิตใน

แบบประกันทรัพย์มิ่งขวัญ 90/15 สามารถลดหย่อนภาษีได้สูงสุด 100,000 บาทตามหลักเกณฑ์ที่กรมสรรพากรกำหนดค่ะ
แบบประกันทรัพย์มิ่งขวัญ 90/15 เหมาะสำหรับผู้สูงอายุ โดยมีจุดเด่นดังนี้:
- คุ้มครองถึงอายุ 90 ปี ครบสัญญารับเงินคืน 150%
- ไม่ต้องตรวจสุขภาพ แต่ต้องแถลงสุขภาพ
- สร้างกองทุนมรดกสำหรับบุตรหลาน
- สามารถลดหย่อนภาษีได้สูงสุด 100,000 บาท

แผนประกันนี้จะช่วยให้คุณได้รับความคุ้มครองในช่วงอายุที่สูงขึ้น และสร้างกองทุนมรดกสำหรับบุตรหลานของคุณได้ด้วย นอกจากนี้ยังสามารถลดหย่อนภาษีได้สูงสุด 100,000 บาทตามกฎหมายที่กำหนดโดยกรมสรรพากรอีกด้วยค่ะ
ไม่จำเป็นต้องมีการตรวจสุขภาพ แต่จำเป็นต้องแถลงสุขภาพ
แผนประกันทรัพย์มิ่งขวัญ 90/15 คุ้มครองถึงอายุ 90 ปีครับ
แบบประกันทรัพย์มิ่งขวัญ 90/15 สามารถลดหย่อนภาษีได้สูงสุดถึง 100,000 บาทตามหลักเกณฑ์ที่กรมสรรพากรกำหนดค่ะ
แผนประกันทรัพย์มิ่งขวัญ 90/15 ไม่มีข้อกำหนดเกี่ยวกับการเปิดเผยสุขภาพ แต่จำเป็นต้องแถลงสุขภาพเพื่อการประกันภัย โดยไม่ต้องตรวจสุขภาพ
แน่นอนครับ แผนประกันทรัพย์มิ่งขวัญ 90/15 ของเราเหมาะสำหรับการสร้างกองทุนมรดกสำหรับลูกหลานของท่านครับ
Exception occurred f

In [12]:
input_file ='Pos_Gen.csv'
OpenAI_engine ='gpt-4'
name_after  ='result'
token = 2000
times = 10
generate_response_nosearch(input_file, prompt,OpenAI_engine,name_after,token,times)

ค่าชดเชยรายวันสำหรับการรักษาในโรงพยาบาลสำหรับแผนประกัน Smile Kids สูงสุดวันละ 3000 บาท สำหรับความคุ้มครองแผนที่ 11.
แผนประกัน Smile Kids มีการชดเชยค่าใช้จ่ายรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลสำหรับ 5 โรคที่พบบ่อยในวัยเด็ก ได้แก่ โรคปอดบวม หรือ ปอดอักเสบ, โรคมือเท้าปาก, โรคอาหารเป็นพิษ, โรคบาดทะยัก, และโรคไข้สมองอักเสบจากเชื้อไวรัส.
ณ วันครบรอบปีกรมธรรม์ที่ 8 ของแผนประกัน Smile Kids, ผู้เอาประกันภัยจะได้รับเงินคืนสูงสุดถึง 40000 บาท.
ในแผนประกัน Smile Kids ณ วันครบกำหนดสัญญา ผู้เอาประกันภัยจะได้รับเงินคืนสูงสุด 600,000 บาท ซึ่งเป็นจำนวนเงินเอาประกันภัยสัญญาหลัก.
ใช่ค่ะ แผนประกัน "สมาย คิดส์" นี้เหมาะสำหรับผู้ปกครองที่ต้องการออมเงินและเรื่องสุขภาพการเจ็บป่วยสำหรับบุตรหลาน รวมถึงผู้ที่ต้องการความคุ้มครองค่ารักษาพยาบาลหากเกิดอุบัติเหตุ สำหรับบุตรหลาน และผู้ที่มีความกังวลเรื่องความเจ็บป่วยของบุตรหลานและสวัสดิการเมื่อต้องเข้ารักษาใน รพ.
แผนประกัน Smile Kids ไม่มีการคุ้มครองค่าใช้จ่ายทางการแพทย์ในกรณีเกิดอุบัติเหตุ แต่มีการคุ้มครองค่าชดเชยรายวันในกรณีเข้าพักรักษาตัวในโรงพยาบาบาลสูงสุดวันละ 

ใช่ค่ะ แบบประกัน P.A. Refund เหมาะสมกับผู้ที่ต้องการความคุ้มครองจากอุบัติเหตุ โดยมีคุ้มครองค่ารักษาพยาบาลจากอุบัติเหตุสูงสุดครั้งละ 100000 บาท และคุ้มครองชีวิตจากอุบัติเหตุสูงสุด 1000000 บาท รวมถึงคุ้มครองกรณีขับขี่หรือโดยสารรถจักรยานยนต์สูงสุด 250000 บาท ด้วยค่ะ
แผนประกัน P.A. Refund มีข้อจำกัดในการคุ้มครองอุบัติเหตุ ดังนี้:
- คุ้มครองค่ารักษาพยาบาลจากอุบัติเหตุสูงสุดครั้งละ 100,000 บาท
- คุ้มครองชีวิตจากอุบัติเหตุสูงสุด 1,000,000 บาท
- คุ้มครองกรณีขับขี่หรือโดยสารรถจักรยานยนต์สูงสุด 250,000 บาท
ทั้งนี้สำหรับความคุ้มครองแผน 3 และอายุผู้เอาประกันภัยต้องอยู่ในช่วง 6-60 ปี
ขออภัยค่ะ แต่ฉันไม่มีข้อมูลเกี่ยวกับแผนประกัน P.A. Refund ที่คุณถามมา กรุณาติดต่อศูนย์บริการลูกค้าของไทยประกันชีวิตที่เบอร์ 1124 (บริการทุกวัน 8:00 - 21:00) หรือส่งอีเมลไปที่ carecenter@thailife.com สำหรับข้อมูลเพิ่มเติมค่ะ.
ขออภัยค่ะ ฉันไม่มีข้อมูลเกี่ยวกับแผนประกัน P.A. Refund ของบริษัทไทยประกันชีวิต ค่ะ หากคุณต้องการข้อมูลเพิ่มเติม กรุณาติดต่อศูนย์บริการลูกค้าของไทยประกันชีวิต ที่เบอร์โทรศัพท์ 1124 บริการทุกวัน ตั้ง

ใช่ค่ะ สำหรับแผนประกันภัย เฮลท์เหมา สบายใจ สบายตังค์ ผู้เอาประกันภัยสามารถหักภาษีจากเบี้ยประกันภัยได้ค่ะ(6) ตามหลักเกณฑ์ที่กรมสรรพากรกำหนดค่ะ
แบบประกันภัยเฮลท์เหมา สบายใจ สบายตังค์ เหมาะสำหรับ:
- ผู้ที่ต้องการความคุ้มครองยามเจ็บป่วย
- พนักงานบริษัทที่มีสวัสดิการสุขภาพอยู่แล้ว
- ผู้ประกอบอาชีพอิสระ ที่ไม่มีสวัสดิการด้านการรักษาพยาบาล
ใช่ค่ะ แผนประกัน "เฮลท์เหมา สบายใจ สบายตังค์" เหมาะสำหรับผู้ที่ต้องการความคุ้มครองค่าใช้จ่ายทางการแพทย์เมื่อเจ็บป่วย โดยมีวงเงินค่ารักษาพยาบาลแบบเหมาจ่ายสูงสุดถึง 5 ล้านบาทต่อรอบปีกรมธรรม์ และไม่ต้องสำรองจ่ายสำหรับโรงพยาบาลตามคู่สัญญาค่ะ.
ใช่ค่ะ แผนประกัน "เฮลท์เหมา สบายใจ สบายตังค์" นี้เหมาะสำหรับผู้ประกอบการที่ไม่มีสิทธิประโยชน์ทางสุขภาพ คุณสามารถสมัครได้โดยไม่ต้องตรวจสุขภาพ แค่ตอบคำถามสุขภาพ 2 ข้อเท่านั้นค่ะ.
ใช่ค่ะ แผนประกันภัย "เฮลท์เหมา สบายใจ สบายตังค์" นี้เหมาะสำหรับผู้ที่มีสวัสดิการสุขภาพจากที่ทำงานแล้ว คุณสามารถเลือกความรับผิดส่วนแรก (Deductible) ได้ด้วยตัวคุณเอง ทำให้สามารถประหยัดค่าเบี้ยประกันภัยได้ค่ะ.
แผนประกัน "เกษียณสุข" เหมาะสำหรับผู้ที่อาย

ใช่ค่ะ, แบบประกัน "อีซี รีฟันด์" มีความคุ้มครองในกรณีเสียชีวิตจากอุบัติเหตุสูงสุด 1,000,000 บาท, ค่าใช้จ่ายทางการแพทย์จากอุบัติเหตุสูงสุด 60,000 บาทต่อครั้ง และค่าชดเชยรายวันเมื่อเข้ารับการรักษาในโรงพยาบาลวันละ 500 บาท ค่ะ.
แน่นอนค่ะ ทั้งสามแบบประกันที่เรามีให้บริการ คือ พี.เอ. รีฟันด์, อีซี รีฟันด์ และ พี.เอ. คลาสสิค ชีลด์ ทั้งหมดนี้มีคุ้มครองกรณีขับขี่หรือโดยสารรถจักรยานยนต์ โดยวงเงินคุ้มครองจะขึ้นอยู่กับแผนที่คุณเลือกค่ะ.
สำหรับแบบประกัน "โรคร้ายแรงรีฟันด์" และ "มะเร็งรีฟันด์" หากไม่มีการเรียกร้องในระยะเวลา 5 ปีติดต่อกัน คุณจะได้รับการคืนเบี้ยประกันภัยปีสุดท้าย 100% ในวันครบกำหนดสัญญา แต่สำหรับแบบประกัน "ออมยาวคุ้มทวีคูณ" ไม่มีข้อกำหนดนี้ค่ะ.
ใช่ค่ะ แบบประกัน:อีซี รีฟันด์ เหมาะสำหรับช่วงอายุ 26-35 ปี และเหมาะสำหรับผู้ที่ต้องการความคุ้มครองจากอุบัติเหตุ มีคุ้มครองกรณีเสียชีวิตจากการเกิดอุบัติเหตุสูงสุด 1,000,000 บาท และคุ้มครองค่ารักษาพยาบาลจากอุบัติเหตุ วงเงินสูงสุด 60,000 บาทต่อครั้ง รวมถึงคุ้มครองกรณีขับขี่หรือโดยสารรถจักรยานยนต์สูงสุด 500,000 บาท ด้วยค่ะ
แบบประกัน: โรคร้ายแรงรีฟั

แบบประกัน Broken Bone จะคุ้มครองค่ารักษาพยาบาลเพิ่มเติมกรณีกระดูกแตกหัก หรือไฟไหม้น้ำร้อนลวกสูงสุด 70000 บาท สำหรับแผน 3.
แบบประกัน: โบรกเกน โบน
จุดเด่นของประกัน:
- คุ้มครองกรณีเสียชีวิตจากการเกิดอุบัติเหตุสูงสุด 1000000 บาท(1) 
- คุ้มครองกรณีสูญเสียอวัยวะสายตา หรือทุพพลภาพถาวรสิ้นเชิง
- คุ้มครองกรณีขับขี่หรือโดยสารรถจักรยานยนต์สูงสุด 500000 บาท(1) 
- คุ้มครองค่ารักษาพยาบาลจากอุบัติเหตุ วงเงินสูงสุด60000 บาทต่อครั้ง(1) 
- คุ้มครองค่ารักษาพยาบาลเพิ่มเติมกรณีกระดูกแตกหัก ไฟไหม้น้ำร้อนลวกสูงสุด 70000 บาท(1) 
ไม่ต้องตรวจสุขภาพ

แบบประกันนี้เหมาะสมกับผู้สูงอายุและบุตรหลานที่ต้องการทำประกันให้พ่อแม่ ดังนั้น แผนประกันนี้เหมาะสมกับพ่อแม่ของคุณครับ.
คุณสามารถซื้อแผนประกัน "โบรกเกน โบน" ได้ครับ แผนประกันนี้เหมาะสำหรับผู้ที่อายุ 50 ปีขึ้นไป และไม่ต้องตรวจสุขภาพก่อนทำประกันครับ
เพื่อให้ได้คำตอบที่ถูกต้องและเหมาะสม คุณสามารถบอกได้ว่าลูกหลานของคุณอายุเท่าไหร่ครับ?
แบบประกัน: Broken Bone
จุดเด่นของประกัน:
- แบบประกันนี้เหมาะสำหรับผู้สูงอายุ
- บุตรหลานที่ต้องการทำประกันให้พ่อแม่
แบบประกัน: เกษียณสุข
จ

แบบประกัน: สมาย คิดส์
จุดเด่นของประกัน:
- ค่าชดเชยรายวัน กรณีเข้าพักรักษาตัวในโรงพยาบาลสูงสุดวันละ 3000 บาท
- ค่าชดเชยรายวัน กรณีเข้าพักรักษาตัว 5 โรคที่พบบ่อยในวัยเด็ก
- ณ วันครบรอบปีกรมธรรม์ที่ 8 รับ เงินคืนสูงสุดถึง 40000 บาท
- ครบกำหนดสัญญารับเงินคืนสูงสุด 600000 บาท

แบบประกันนี้เหมาะสำหรับผู้ที่ต้องการความคุ้มครองค่ารักษาพยาบาลหากเกิดอุบัติเหตุ สำหรับบุตรหลาน และผู้ที่มีความกังวลเรื่องความเจ็บป่วยของบุตรหลานและสวัสดิการเมื่อต้องเข้ารักษาใน รพ.


In [18]:
input_file ='Pos_Gen2.csv'
OpenAI_engine ='gpt-4'
name_after  ='result'
token = 2000
times = 10
generate_response_nosearch(input_file, prompt,OpenAI_engine,name_after,token,times)

สำหรับแผนประกัน Smile Kids ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลสูงสุดวันละ 3000 บาท สำหรับความคุ้มครองแผนที่ 11 ค่ะ.
แบบประกัน Smile Kids จะชดเชยค่าใช้จ่ายในโรงพยาบาลประจำวันสูงสุดวันละ 3000 บาท สำหรับการเข้าพักรักษาตัว 5 โรคที่พบบ่อยในวัยเด็ก ตามแผนที่ 11 ค่ะ.
แผนประกัน Smile Kids สามารถรับเงินคืนสูงสุดถึง 600,000 บาท เมื่อครบกำหนดสัญญา และสูงสุดถึง 40,000 บาท ณ วันครบรอบปีกรมธรรม์ที่ 8 โดยขึ้นอยู่กับจำนวนเงินเอาประกันภัยสัญญาหลัก.
Error: The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the error persists. (Please include the request ID 867fe8f4-120a-4826-ad57-64b9b6f65a21 in your message.) {
  "error": {
    "message": "The server had an error while processing your request. Sorry about that! You can retry your request, or contact us through an Azure support request at: https://go.microsoft.com/fwlink/?linkid=2213926 if the e

สำหรับแผนประกัน P.A. Classic Shield คุณไม่สามารถเพิ่มความคุ้มครองได้เองครับ แต่คุณสามารถติดต่อศูนย์บริการลูกค้าของไทยประกันชีวิตที่เบอร์ 1124 หรือส่งอีเมลไปที่ carecenter@thailife.com เพื่อขอคำปรึกษาเพิ่มเติมครับ.
แผนประกันภัย P.A. Beyond Smart Protection มีการคุ้มครองสูงสุด 500,000 บาท ในกรณีเสียชีวิตจากอุบัติเหตุ สำหรับแผน 3.
แบบประกัน: พี.เอ. บียอนด์ สมาร์ตโพรเทกชั่น จะมีวงเงินค่ารักษาพยาบาลจากอุบัติเหตุสูงสุด 50000 บาท สำหรับแผน 3.
เบี้ยประกันภัยของแผนประกันภัย P.A. Beyond Smart Protection เริ่มต้นที่วันละ 8 บาทค่ะ.
แผนประกันภัย P.A. Beyond Smart Protection จะให้เงินชดเชยรายวันจากอุบัติเหตุให้ วันละ 1000 บาท ครับ
แบบประกัน:พี.เอ. รีฟันด์ คุ้มครองกรณีขับขี่หรือโดยสารรถจักรยานยนต์สูงสุด 250000 บาท(1)

แบบประกัน:พี.เอ. คลาสสิค ชีลด์ คุ้มครองกรณีขับขี่หรือโดยสารรถจักรยานยนต์สูงสุด 600000 บาท(1)

แบบประกัน:อีซี รีฟันด์ คุ้มครองกรณีขับขี่หรือโดยสารรถจักรยานยนต์สูงสุด 500000 บาท(1)

(1)สำหรับความคุ้มครองแผน 3
แบบประกัน: พี.เอ. บียอนด์ สมาร์ตโพรเทกชั่น เหมาะสำหรับช่วงอายุ 0-25 ปี

แบบประกั

แบบประกัน: ทุนทวีพลัส 20/20 
จุดเด่นของประกัน:
- ชำระเบี้ยฯ 20 ปี คุ้มครอง 20 ปี 
- รับเงินคืนในวันครบรอบ ปีกรมธรรม์ที่ 51015 รับเงินคืน 8% 
- เงินคืนตลอดสัญญารวม 174% 
- รับเงินชดเชยกรณีรักษาตัวในโรงพยาบาล500 บาท/วัน 
- รับความคุ้มครองจากอุบัติเหตุเพิ่มสูงสุด 100000 บาท 
- ลดหย่อนภาษีได้สูงสุด100000 บาท 

แบบประกันนี้เหมาะกับผู้ที่ต้องการมีหลักประกันที่มั่นคงในชีวิต ด้วยการชำระเบี้ยประกันภัยในระยะเวลา 20 ปี และได้รับความคุ้มครองในระยะเวลาเดียวกัน รวมถึงการรับเงินคืนในวันครบรอบปีกรมธรรม์และเงินคืนตลอดสัญญา และยังมีสิทธิ์ในการลดหย่อนภาษีได้ด้วย
แผนประกันทุนทวีพลัส 20/20 จะต้องชำระเบี้ยประกันภัยเป็นเวลา 20 ปีค่ะ.
แผนประกันทุนทวีพลัส 20/20 มีการคุ้มครองเป็นเวลา 20 ปีค่ะ.
แผนประกันทุนทวีพลัส 20/20 ไม่มีการคืนเงินในวันครบรอบประกันภัยปีที่ 5, 10, 15 แต่จะมีการคืนเงินในวันครบรอบประกันภัยที่ 5, 10, 15 โดยรับเงินคืน 8% ของจำนวนเงินเอาประกันภัย.
แผนประกันภัย "แฮปปี้มีเงินใช้" เหมาะสำหรับผู้ที่อายุระหว่าง 26-50 ปีค่ะ.
ในแผนประกันภัย แฮปปี้มีเงินใช้ คุณจะได้รับเงินคืน 3% ของจำนวนเงินเอาประกันภัยใน

ใช่ค่ะ คุณสามารถใช้สิทธิ์หักลดภาษีจากแผนประกันภัย "มะเร็งรีฟันด์" ได้ค่ะ
ความคุ้มครองสูงสุดสำหรับการตรวจพบมะเร็งทุกชนิด (ยกเว้นมะเร็งผิวหนัง) ในแผนประกันภัย มะเร็ง รีฟันด์ คือ 1,000,000 บาท สำหรับแผน 3.
ค่าชดเชยรายวันสำหรับการนอนโรงพยาบาลจากแผนประกันภัย มะเร็ง รีฟันด์ สูงสุดคือ 1000 บาทต่อวัน และสูงสุดไม่เกิน 365 วัน.
แบบประกัน "มะเร็งรีฟันด์" ของไทยประกันชีวิตเหมาะสำหรับผู้ที่อายุระหว่าง 26-50 ปีค่ะ.
ใช่ค่ะ คุณสามารถใช้แผนประกันภัย "มะเร็งรีฟันด์" ในการหักลดภาษีได้ค่ะ
ในแผนประกันภัย Easy Family คุณสามารถรับค่าชดเชยในกรณีที่เสียชีวิตจากอุบัติเหตุได้สูงสุด 500,000 บาท สำหรับแผนความคุ้มครองที่ 4
แบบประกันภัย Easy Family คุ้มครองค่ารักษาพยาบาลจากอุบัติเหตุสูงสุด 10,000 บาทต่อครั้งค่ะ.
ในแผนประกันภัย Easy Family คุณสามารถรับค่าชดเชยรายวันในระหว่างการรักษาพยาบาลในโรงพยาบาลได้สูงสุดวันละ 500 บาท.
แผนประกันภัย Easy Family มีความคุ้มครองสำหรับการขับขี่หรือโดยสารรถจักรยานยนต์สูงสุด 250,000 บาท สำหรับแผน 4.
แบบประกันภัย Easy Family มีการคุ้มครองสำหรับครอบครัว และเมื่อสมัครทั้งครอบครัว ค่าเบี้ยปร

เบี้ยประกันภัยของแผนประกัน "สูงวัยมีทรัพย์" เริ่มต้นเพียงวันละ 9 บาท
ขออภัยค่ะ ฉันไม่มีข้อมูลเกี่ยวกับแผนประกัน "สูงวัยมีทรัพย์" ของบริษัทไทยประกันชีวิต ค่ะ หากคุณต้องการข้อมูลเพิ่มเติม กรุณาติดต่อศูนย์บริการลูกค้าของไทยประกันชีวิต ที่เบอร์โทรศัพท์ 1124 หรือทางอีเมลที่ carecenter@thailife.com ค่ะ
แผนประกัน "สูงวัยมีทรัพย์" ไม่ต้องการการตรวจสุขภาพและไม่ต้องแถลงสุขภาพค่ะ.
แบบประกัน:สูงวัยมีทรัพย์ เหมาะสำหรับผู้สูงอายุโดยเฉพาะ และผู้ที่ต้องการหาความคุ้มครองสำหรับผู้สูงอายุ รวมถึงผู้สูงอายุที่ต้องการมีมรดกให้ลูกหลาน
ใช่ค่ะ แผนประกัน "สูงวัยมีทรัพย์" ถูกออกแบบมาเพื่อผู้สูงอายุโดยเฉพาะ และเหมาะสำหรับผู้ที่อายุระหว่าง 50-70 ปี ไม่ต้องตรวจสุขภาพและไม่ต้องแถลงสุขภาพค่ะ
ใช่ค่ะ แผนประกัน "สูงวัยมีทรัพย์" ถูกออกแบบมาเพื่อผู้สูงอายุโดยเฉพาะ โดยมีคุณสมบัติดังนี้:
- วันครบรอบปีกรมธรรม์ที่ผู้เอาประกันภัยมีอายุ 75 ปีรับ 50% ของจำนวนเงินเอาประกันภัยสูงถึง 200000 บาท
- คุ้มครองชีวิต จากการเจ็บป่วยและจากอุบัติเหตุ สูงถึง 600000 บาท
- ครบสัญญารับเงินคืนสูงถึง 600000 บาท
- เบี้ยฯ เริ่มต้น เพียงวันละ 9 บาท
-

แบบประกัน: พี.เอ. คลาสสิค ชีลด์
จุดเด่นของประกัน:
- คุ้มครองอุบัติเหตุทั่วไป 24 ชั่วโมงทั่วโลก
- ค่ารักษาพยาบาลกรณีอุบัติเหตุ
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาล
- ค่าปลงศพกรณีเสียชีวิตจากอุบัติเหตุ
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคร้ายแรง
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคทั่วไป
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคร้ายแรง
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคทั่วไป
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคร้ายแรง
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคทั่วไป
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคร้ายแรง
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคทั่วไป
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคร้ายแรง
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคทั่วไป
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคร้ายแรง
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคทั่วไป
- ค่าชดเชยรายวันกรณีเข้าพักรักษาตัวในโรงพยาบาลจากโรคร้ายแรง
- ค่าชดเชยรายวัน